## Building a Logistic Regression Model From Scratch

In [1]:
import pandas as pd
import numpy as np
import os 
from sklearn.linear_model import LogisticRegression

Methods:

1. An `__init__()` method that takes in an error tolerance as a stopping criterion, as well as max number of iterations.
2. A `predict_proba()` method that takes a given matrix of features $X$ and predicts $P = \dfrac{1}{1+e^{-(X \cdot W+\alpha)}}$ for each entry
3. A `compute_gradient()` method that computes the gradient vector $G$
4. A `compute_hessian()` method that computes the Hessian. Note that the $H$ can be broken down to the following matrix multiplications: $H=(X^T*Q)\cdot X$. 
5. An `update_weights()` method that applies gradient descent to update the weights
6. A `check_stop()` method that checks whether the model has converged or the max iterations have been met
7. A `fit()` method that trains the model. It takes in the data and runs the gradient optimization

In [2]:
class LogisticRegressionScratch(object):
    
    def __init__(self, tolerance = 10**-8, max_iterations = 20):
        
        self.tolerance = tolerance
        self.max_iterations = max_iterations
        self.weights_array = None # holds current weights and intercept (intercept is at the last position)
        self.prior_w = None # holds previous weights and intercept (intercept is at the last position)
        
        # once we are done training, these variables will hold the 
        # final values for the weights and intercept
        self.weights = None
        self.intercept = None 

        
    def predict_proba(self, X):
        '''
        Compute probabilities using the inverse logit
        - Inputs: The Nx(K+1) matrix with intercept column X
        - Outputs: Vector of probabilities of length N
        '''
        
        XW = np.dot(X, self.weights_array)
        P = 1 / (1+np.exp(-XW))
        
        
        return P

    
    
    def compute_gradient(self, X, y, P):
        '''
        Computes the gradient vector
        -Inputs:
            - The Nx(K+1) matrix with intercept column X
            - Nx1 vector y (label) 
            - Nx1 vector of predictions P
        -Outputs: 1x(K+1) vector of gradients
        '''
        
        G = -np.dot(X.T, (y-P))

                
        return G
        
    def compute_hessian(self, X, P):
        '''
        computes the Hessian matrix
        -inputs:
            - Nx(K+1) matrix X
            - Nx1 vector of predictions P
        -outputs:
            - KxK Hessian matrix H=X^T * Diag(Q) * X
        '''

        Q = P*(1-P)
        XQ = X.T * Q
        H = np.dot(XQ, X)
        
        return H


    def update_weights(self, X, y):
        '''
        Updates existing weight vector
        -Inputs:
            -Nx(Kx1) matrix X
            -Nx1 vector y
        -Calls predict_proba, compute_gradient and compute_hessian and uses the 
        return values to update the weights array
        '''
        
        P = self.predict_proba(X)
        G = self.compute_gradient(X, y, P)
        H = self.compute_hessian(X, P)
        self.prior_w = np.copy(self.weights_array)
        H_inv = np.linalg.inv(H)
        update_step = np.dot(H_inv, G)
        self.weights_array-=update_step
        
      
           
    def check_stop(self):
        '''
        check to see if euclidean distance between old and new weights (normalized)
        is less than the tolerance
        
        returns: True or False on whether stopping criteria is met
        '''
        
        w_old_norm = self.prior_w / np.linalg.norm(self.prior_w)
        w_new_norm = self.weights_array / np.linalg.norm(self.weights_array)
        diff = w_old_norm - w_new_norm
        distance = np.sqrt(np.dot(diff, diff))
        stop = distance < self.tolerance
        
        
        return stop
        
        
    def fit(self, X, y):
        '''
        X is the Nx(K-1) data matrix
        Y is the labels, using {0,1} coding
        '''
        
        #set initial weights - add an extra dimension for the intercept
        self.weights_array = np.zeros(X.shape[1] + 1)
        
        #Initialize the slope parameter to log(base rate/(1-base rate))
        self.weights_array[-1] = np.log(y.mean() / (1-y.mean()))
        
        #create a new X matrix that includes a column of ones for the intercept
        X_int = np.hstack((X, np.ones((X.shape[0],1))))

        for i in range(self.max_iterations):
            self.update_weights(X_int, y)
            
            # check whether we should
            stop = self.check_stop()
            if stop:
                # since we are stopping, lets save the final weights and intercept
                self.set_final_weights()
                self.set_final_intercept()
                break
                
    
    def set_final_weights(self):
        self.weights = self.weights_array[0:-1]
        
    def set_final_intercept(self):
        self.intercept = self.weights_array[-1]  
        
    def get_weights(self):
        return self.weights
    
    def get_intercept(self):
        return self.intercept
        

#### Use the Class to Train a Logistic Regression Model

In [3]:
filename = os.path.join(os.getcwd(), "data", "airbnbData_train.csv")
df = pd.read_csv(filename, header = 0)

In [4]:
feature_list = ['review_scores_rating','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_value','host_response_rate','host_acceptance_rate']
feature_list

['review_scores_rating',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_value',
 'host_response_rate',
 'host_acceptance_rate']

In [5]:
y = df["host_is_superhost"]
X = df[feature_list]

In [6]:
lr = LogisticRegressionScratch()
lr.fit(X, y)
print('The fitted weights and intercept are:')
print(lr.get_weights(), lr.get_intercept())

The fitted weights and intercept are:
[ 0.56690005  0.492255    0.201587    0.25551467 -0.00590516  1.71592957
  0.26478817] -1.829062262272181


#### Compare with Scikit-Learn's Implementation

In [7]:
lr_sk = LogisticRegression(C=10**10)
lr_sk.fit(X, y)

LogisticRegression(C=10000000000)

In [8]:
print('The fitted weights and intercept with sklearn are:')
print(lr_sk.coef_, lr_sk.intercept_)


The fitted weights and intercept with sklearn are:
[[ 5.61260835e-01  4.93119692e-01  2.00796873e-01  2.57547131e-01
  -1.57860144e-03  1.71834907e+00  2.64799558e-01]] [-1.82994455]


In [9]:
# %timeit to fit the logistic model lr on the training data
%timeit lr.fit(X, y)

# %timeit to fit the logistic model lr_sk on the training data
%timeit lr_sk.fit(X, y)

6.19 ms ± 1.14 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
13.8 ms ± 498 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
